In [1]:
from rich import print as rprint
from rich.style import Style

In [2]:
import random
from IPython.display import Markdown, display
import os
import google.generativeai as genai

from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

# *** IMPORTANTE: Configure sua chave de API do Google AI Studio aqui ***
# Você pode definir como variável de ambiente (recomendado):
# Vá em "Ambiente de execução" -> "Variáveis de ambiente" e adicione
# uma variável com nome: GOOGLE_API_KEY
# e valor: SUA_CHAVE_DE_API

# Ou, se preferir (MENOS SEGURO para compartilhar o notebook):
# genai.configure(api_key="SUA_CHAVE_DE_API_AQUI")

# Se a variável de ambiente estiver configurada, a linha abaixo irá carregá-la:
#genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Seleciona o modelo gemini-2.0-flash
model = genai.GenerativeModel('gemini-2.0-flash')

In [3]:
estilos_arte = [
    "Realismo", "Surrealismo", "Impressionismo", "Expressionismo", "Cubismo",
    "Fauvismo", "Art Nouveau", "Art Deco", "Barroco", "Rococó",
    "Gótico", "Renascimento", "Romantismo", "Abstracionismo", "Pop Art",
    "Op Art", "Minimalismo", "Bauhaus", "Dadaísmo", "Futurismo",
    "Simbolismo", "Pointilhismo", "Hiper-realismo", "Cartoon", "Mangá",
    "Anime dos anos 90", "Steampunk", "Cyberpunk", "Vaporwave", "Synthwave",
    "Pixel Art", "Low Poly", "Ilustração Infantil", "Aquarela", "Óleo sobre tela",
    "Grafite", "Escultura", "Fotografia", "Arte conceitual", "Bobbie Goods",
    "Ukiyo-e", "Arte bizantina", "Arte egípcia", "Arte grega antiga", "Arte romana",
    "Arte medieval", "Renascimento nórdico", "Neoclassicismo", "Pós-impressionismo"
]

In [4]:
def apresentar_robo_berry():
    apresentacao = """
<div style="background-color:#f0f8ff; padding: 15px; border-radius: 10px;">
    <h2 style="color:#4682B4;">🤖 Olá! Eu sou o Robô Berry!</h2>
    <p>Ainda não consigo gerar uma imagem para você baixar, mas consigo te ajudar a criar <b>ótimos prompts</b> (ordens) para que você envie no meu parceiro <a href="https://gemini.google.com" target="_blank" style="color:#008080; text-decoration: none;">gemini.google.com</a> e ele crie essa imagem para você!</p>
    <p><b>Como funciona?</b> 🤔 Vou te fazer algumas perguntas e, no final, entregarei dois textos (prompts)! Um baseado nas suas respostas e outro sugerido por IA! Você pode copiar e colar qualquer um deles no link que vou deixar para você.</p>
    <p><span style="color:#FFA07A;">💡 Dica:</span> Caso não saiba a resposta, pode apenas deixar em branco, eu me vira com o que tenho. A parte de "<span style="color:#9932CC;">Sentimento</span>" é muito legal! Se possível, sempre preencha ela.</p>
    <p style="color:#228B22;">Ficarei feliz em ajudar! 😄</p>
</div>
"""
    display(Markdown(apresentacao))

In [5]:
def gerar_prompt():
    respostas = {}

    def perguntar(numero, pergunta, alternativas=None):
        rprint(f"\n[bold blue]{numero})[/bold blue] [bold]{pergunta}[/bold]")
        if alternativas:
            for letra, opcao in alternativas.items():
                rprint(f"   [bold green]{letra})[/bold green] {opcao}")
            resposta = input("Sua resposta (pela letra ou digite): ").strip()
            if resposta.lower() in alternativas:
                return resposta, alternativas[resposta.lower()]
            else:
                return resposta, resposta
        else:
            resposta = input("Sua resposta: ").strip()
            return resposta, resposta

    respostas["tema"], _ = perguntar(1, "Tema central da imagem.\nEx.: Uma paisagem calma; um personagem mitológico; Um sentimento abstrato; etc..")
    print() # Adiciona espaço
    respostas["ambientacao"], _ = perguntar(2, "Ambientação. Onde essa imagem está?\nEx.: \"Um castelo abandonado\"; \"Uma sala escura\"; \"Um restaurante futurista\"; etc..")
    print() # Adiciona espaço
    exemplos_estilos = random.sample(estilos_arte, min(3, len(estilos_arte)))
    pergunta_estilo = f"Estilo artístico da imagem.\nEx.: {exemplos_estilos[0]}, {exemplos_estilos[1]}, {exemplos_estilos[2]}"
    respostas["estilo"], _ = perguntar(3, pergunta_estilo)
    print() # Adiciona espaço
    respostas["finalidade"], _ = perguntar(4, "Finalidade da imagem.\nEx.: \"Enviar por aplicativo de mensagem\"; \"Imprimir em tamanho A4\"; \"Logo para meu negócio\".")
    print() # Adiciona espaço
    respostas["sentimento"], _ = perguntar(5, "Sentimento. Qual sentimento essa imagem deve transmitir?\nEx.: \"Felicidade\"; \"Angústia\"; \"Esperança\".")
    print() # Adiciona espaço
    respostas["iluminacao"], _ = perguntar(6, "Iluminação. (Se não souber, pode deixar em branco que eu me vira!)\nEx.: \"Luz matinal\"; \"Luz artificial\"; \"Tons de verde\"")
    print() # Adiciona espaço
    respostas["paleta_cores"], _ = perguntar(7, "Paleta de cores.\nEx.: \"Tons pastéis\"; \"Neon\"; \"Cores vibrantes\".")
    print() # Adiciona espaço
    respostas["texto"], _ = perguntar(8, "Estamos quase no final!! Deseja algum TEXTO? O ideal são frases de até 10 palavras. (Essa é uma função bem difícil para o meu amigo executar, talvez o resultado não saia como o esperado)\nEx.: \"Bom dia! Que o dia seja maravilhoso para você!\"; \"VAMO PARA CIMA! HADOUKEN!!!\"; etc..")
    print() # Adiciona espaço
    alternativas_composicao = {
        "a": "Frontal",
        "b": "De cima",
        "c": "Lateral",
        "d": "Close-up",
        "e": "Plano aberto"
    }
    resposta_composicao, texto_composicao = perguntar(9, "Composição, ângulo e enquadramento.", alternativas_composicao)
    respostas["composicao"] = resposta_composicao
    respostas["composicao_texto"] = texto_composicao
    print() # Adiciona espaço
    respostas["restricao"], _ = perguntar(10, "Faltou algo? Tem alguma restrição? Pode colocar aqui!\nEx.: \"Não quero fadas na minha imagem\"; \"Incluir um pouco de glitter em todas as superfícies\"; \"Quero uma imagem em baixíssima qualidade\"; etc..")
    print() # Adiciona espaço

    return respostas

In [6]:
from rich import print as rprint
from rich.style import Style

def formatar_prompt(respostas):
    prompt_original = ""
    prompt_original += f"Crie uma imagem de {respostas['tema']}.\n"
    prompt_original += f"A ambientação deve ser em {respostas['ambientacao']}.\n"
    prompt_original += f"O estilo é {respostas['estilo']}.\n"
    prompt_original += f"A finalidade é para {respostas['finalidade']}.\n"
    prompt_original += f"O sentimento é de {respostas['sentimento']}.\n"
    prompt_original += f"A iluminação é {respostas['iluminacao'] if respostas['iluminacao'] else 'não foi definida'}.\n"
    prompt_original += f"A paleta de cores é {respostas['paleta_cores'] if respostas['paleta_cores'] else 'não foi definida'}.\n"
    prompt_original += f"Texto: \"{respostas['texto']}\"." if respostas['texto'] else ""
    prompt_original += f"\nO enquadramento da imagem é {respostas['composicao_texto']}." if 'composicao_texto' in respostas else ""
    prompt_original += f"\nConsidere que {respostas['restricao']}." if respostas['restricao'] else ""

    prompt_ia = chamar_api_ia(prompt_original)

    prompt_formatado_markdown = ""
    for linha in prompt_original.split('\n'):
        prompt_formatado_markdown += f"**<span style='color:gold;'>{linha}</span>**<br>\n"

    mensagem_final = f"""
<div style="background-color:#FFFACD; padding: 15px; border-radius: 10px;">
    <h2 style="color:#DAA520;">✨ Seus Prompts para Comparação! ✨</h2>
    <h3 style="color:#DAA520;">Prompt Original (Baseado nas suas escolhas):</h3>
    <p style="font-size: 1em; line-height: 1.4;">{prompt_formatado_markdown}</p>
    <hr style="border-top: 1px dashed #DAA520;">
    <h3 style="color:#8FBC8F;">Prompt Sugerido pela IA:</h3>
    <pre style="font-size: 1em; line-height: 1.4;">{prompt_ia}</pre>
    <hr style="border-top: 1px dashed #DAA520;">
    <p><b>Passo a passo:</b></p>
    <ol>
        <li>Copie o prompt desejado (Original ou IA) e cole no <a href="https://gemini.google.com" target="_blank" style="color:#008080; text-decoration: none;">gemini.google.com</a> para gerar a imagem.</li>
    </ol>
</div>
"""
    display(Markdown(mensagem_final))
    rprint(f"[bold green]Prompt da IA:[/bold green]\n{prompt_ia}")
    return prompt_original, prompt_ia

In [7]:
def chamar_api_ia(prompt_usuario):
    """
    Chama o modelo Gemini Pro para gerar um prompt alternativo.
    """
    if model is None:
        return "Erro: Modelo de IA não carregado."

    prompt_de_geracao = f"""
    Analise o seguinte prompt para geração de imagens:
    '{prompt_usuario}'
    Gere um prompt alternativo que possa produzir uma imagem visualmente interessante,
    mantendo a essência do tema e do sentimento, mas talvez explorando um estilo artístico diferente
    ou adicionando detalhes de iluminação e composição mais específicos.
    O prompt gerado deve ser conciso e claro para um gerador de imagens de IA.
    """
    try:
        response = model.generate_content(prompt_de_geracao)
        prompt_ia_gerado = response.text
        return prompt_ia_gerado
    except Exception as e:
        return f"Erro ao gerar prompt pela IA: {e}"

In [8]:
def robo_berry():
    historico_prompts = []
    while True:
        respostas = gerar_prompt()
        prompt_original, prompt_ia = formatar_prompt(respostas)
        historico_prompts.append((prompt_original, prompt_ia)) # Salvar ambos os prompts

        print("\nGostou? Quer fazer mais?")
        print("a) Fazer outro prompt.")
        print("b) Ver histórico de prompts.")
        print("c) Sair.")

        escolha = input("Sua escolha: ").lower()
        if escolha == 'a':
            continue
        elif escolha == 'b':
            if historico_prompts:
                print("\n✨ Seu Histórico de Prompts: ✨")
                for i, (original, ia) in enumerate(historico_prompts):
                    print(f"\n{i+1}) Prompt Original:\n{original}\n   Prompt IA:\n{ia}")
            else:
                print("\nSeu histórico está vazio!")
        elif escolha == 'c':
            print("\nFoi ótimo te ajudar! Se precisar de mais alguma coisa, é só chamar! 😊")
            break
        else:
            print("\nOpção inválida. Por favor, escolha 'a', 'b' ou 'c'.")

# Iniciar o Robô Berry
apresentar_robo_berry()
robo_berry()


<div style="background-color:#f0f8ff; padding: 15px; border-radius: 10px;">
    <h2 style="color:#4682B4;">🤖 Olá! Eu sou o Robô Berry!</h2>
    <p>Ainda não consigo gerar uma imagem para você baixar, mas consigo te ajudar a criar <b>ótimos prompts</b> (ordens) para que você envie no meu parceiro <a href="https://gemini.google.com" target="_blank" style="color:#008080; text-decoration: none;">gemini.google.com</a> e ele crie essa imagem para você!</p>
    <p><b>Como funciona?</b> 🤔 Vou te fazer algumas perguntas e, no final, entregarei dois textos (prompts)! Um baseado nas suas respostas e outro sugerido por IA! Você pode copiar e colar qualquer um deles no link que vou deixar para você.</p>
    <p><span style="color:#FFA07A;">💡 Dica:</span> Caso não saiba a resposta, pode apenas deixar em branco, eu me vira com o que tenho. A parte de "<span style="color:#9932CC;">Sentimento</span>" é muito legal! Se possível, sempre preencha ela.</p>
    <p style="color:#228B22;">Ficarei feliz em ajudar! 😄</p>
</div>


1) Tema central da imagem.
Ex.: Uma paisagem calma; um personagem mitológico; Um sentimento abstrato; etc..

Sua resposta: cachorro preto



2) Ambientação. Onde essa imagem está?
Ex.: "Um castelo abandonado"; "Uma sala escura"; "Um restaurante futurista"; etc..

Sua resposta: restaurante



...


<div style="background-color:#FFFACD; padding: 15px; border-radius: 10px;">
    <h2 style="color:#DAA520;">✨ Seus Prompts para Comparação! ✨</h2>
    <h3 style="color:#DAA520;">Prompt Original (Baseado nas suas escolhas):</h3>
    <p style="font-size: 1em; line-height: 1.4;">**<span style='color:gold;'>Crie uma imagem de cachorro preto.</span>**<br>
**<span style='color:gold;'>A ambientação deve ser em restaurante.</span>**<br>
**<span style='color:gold;'>O estilo é ukiyo-e.</span>**<br>
**<span style='color:gold;'>A finalidade é para enviar por wpp.</span>**<br>
**<span style='color:gold;'>O sentimento é de esperança.</span>**<br>
**<span style='color:gold;'>A iluminação é não foi definida.</span>**<br>
**<span style='color:gold;'>A paleta de cores é não foi definida.</span>**<br>
**<span style='color:gold;'>Texto: "vai agora".</span>**<br>
**<span style='color:gold;'>O enquadramento da imagem é Frontal.</span>**<br>
</p>
    <hr style="border-top: 1px dashed #DAA520;">
    <h3 style="color:#8FBC8F;">Prompt Sugerido pela IA:</h3>
    <pre style="font-size: 1em; line-height: 1.4;">**Prompt Alternativo:**

"Cachorro preto estilizado em aquarela, com olhar esperançoso, sentado à mesa de um restaurante aconchegante. Iluminação suave e difusa, vinda de uma janela lateral. Paleta de cores quentes e terrosas (tons de sépia, ocre, marrom e toques de vermelho). Texto flutuante em caligrafia cursiva: 'Vai Agora'. Enquadramento frontal, close-up no rosto do cachorro."
</pre>
    <hr style="border-top: 1px dashed #DAA520;">
    <p><b>Passo a passo:</b></p>
    <ol>
        <li>Copie o prompt desejado (Original ou IA) e cole no <a href="https://gemini.google.com" target="_blank" style="color:#008080; text-decoration: none;">gemini.google.com</a> para gerar a imagem.</li>
    </ol>
</div>


Prompt da IA:
**Prompt Alternativo:**

"Cachorro preto estilizado em aquarela, com olhar esperançoso, sentado à mesa de um restaurante aconchegante. 
Iluminação suave e difusa, vinda de uma janela lateral. Paleta de cores quentes e terrosas (tons de sépia, ocre, 
marrom e toques de vermelho). Texto flutuante em caligrafia cursiva: 'Vai Agora'. Enquadramento frontal, close-up 
no rosto do cachorro."


Gostou? Quer fazer mais?
a) Fazer outro prompt.
b) Ver histórico de prompts.
c) Sair.
Sua escolha: b

✨ Seu Histórico de Prompts: ✨

1) Prompt Original:
Crie uma imagem de cachorro preto.
A ambientação deve ser em restaurante.
O estilo é ukiyo-e.
A finalidade é para enviar por wpp.
O sentimento é de esperança.
A iluminação é não foi definida.
A paleta de cores é não foi definida.
Texto: "vai agora".
O enquadramento da imagem é Frontal.
   Prompt IA:
**Prompt Alternativo:**

"Cachorro preto estilizado em aquarela, com olhar esperançoso, sentado à mesa de um restaurante aconchegante. Iluminação suave e difusa, vinda de uma janela lateral. Paleta de cores quentes e terrosas (tons de sépia, ocre, marrom e toques de vermelho). Texto flutuante em caligrafia cursiva: 'Vai Agora'. Enquadramento frontal, close-up no rosto do cachorro."



1) Tema central da imagem.
Ex.: Uma paisagem calma; um personagem mitológico; Um sentimento abstrato; etc..

KeyboardInterrupt: Interrupted by user